Rebuilding Steps, Example Network, Integrated in SNMan 'Multi' Rebuild
====================
xxx

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import copy
import os
import sys
import networkx as nx
import numpy as np
import shutil

import snman
from snman.constants import *
from snman import osmnx_customized as oxc
from snman import *

#PERIMETER = '_paper2_example'
PERIMETER = '_troubleshooting'
#PERIMETER = 'zrh_north-west'
#PERIMETER = 'ebc_zrh_v01_10km'

data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
export_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

Loading data
------------
Loads the prepared simplified street graph, as well as other datasets such as perimeters, rebuilding regions and measurement regions

In [ ]:
print('Load street graph')
G = snman.io.load_street_graph(
    os.path.join(export_path, 'G_edges_given.gpkg'),
    os.path.join(export_path, 'G_nodes_given.gpkg'),
    crs=CRS_internal
)

street_graph.organize_edge_directions(G)

In [ ]:
if 1:
    
    print('Load rebuilding regions')
    # Polygons that define which streets will be reorganized
    rebuilding_regions_gdf = snman.io.load_rebuilding_regions(
        os.path.join(inputs_path, 'rebuilding_regions', 'rebuilding_regions.gpkg'),
        #filter_ids=['zrh-6'],
        #filter_ids=['zrh-main'],
        #filter_ids=['playground-1'],
        filter_ids=['zrh-61'],
        crs=CRS_internal
    )
    
    print('Load access needs')
    access_needs = snman.io.load_access_needs(
        os.path.join(inputs_path, 'switzerland', 'zurich', 'statpop', 'statpop17_zurich_city_agg_on_buildings.gpkg'),
        CRS_internal
    )
    
    access_needs['parking_spots_needed'] = access_needs['residents'] / 60
    
    import geopandas as gpd
    from snman import *
    from snman.rebuilding import *
    
    HLAs = multi_rebuild_regions(
        G, rebuilding_regions_gdf, access_needs,
        save_steps_path=os.path.join(export_path, 'steps'),
        #source_lanes_attribute=KEY_LANES_DESCRIPTION_AFTER,
        #target_lanes_attribute=KEY_LANES_DESCRIPTION_AFTER,
        gravity_iterations=3
    )
    HLAs = list(filter(lambda HLA: HLA is not None, HLAs))

In [ ]:
L = snman.io.load_lane_graph(
    os.path.join(export_path, 'steps', '3_L_edges.gpkg'),
    os.path.join(export_path, 'steps', '3_L_nodes.gpkg')
)

In [ ]:
H = copy.deepcopy(G)

In [ ]:
snman.io.export_HLA(
    os.path.join(export_path, 'steps'),
    '4',
    L=L, H=H
)

In [ ]:
snman.rebuilding._remove_cycling_lanes(L, None, H, 'width', verbose=True)

snman.io.export_HLA(
    os.path.join(export_path, 'steps'),
    '4',
    L=L, H=H
)

In [ ]:
snman.rebuilding._remove_car_lanes(L, None, H, 'width', None, mode='highest_bc', stop_at_step=5000, verbose=True)

snman.io.export_HLA(
    os.path.join(export_path, 'steps'),
    '4b',
    L=L, H=H
)

In [ ]:
snman.rebuilding.check_if_edge_breaks_transit(H, L, 39641, 28490, '28490-39641-0-0')

In [ ]:
snman.rebuilding.check_if_edge_breaks_transit(H, L, 28490, 39641, '28490-39641-0-2')

In [ ]:
snman.rebuilding.check_if_edges_disconnect_graph(
    L, [(29276,29277,'29276-29277-0-0')],
    'needs_access_by_private_cars'
)

In [ ]:
snman.lane_graph.get_dependent_parking_lanes(L, 25079, 32916, '25079-32916-0-0')

In [ ]:
snman.lane_graph.get_dependent_parking_lanes(L, 27266, 30527, '27266-30527b-0-0')

In [ ]:
print('Reorder lanes')
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION)
snman.space_allocation.reorder_lanes(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)

print('Normalize cycling infrastructure')
snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION)
snman.space_allocation.normalize_cycling_lanes(G, lanes_key=KEY_LANES_DESCRIPTION_AFTER)

snman.street_graph.organize_edge_directions(G)

print('export G')
snman.io.export_street_graph(
    G,
    os.path.join(export_path, 'G_edges.gpkg'),
    os.path.join(export_path, 'G_nodes.gpkg')
)

print('export L after')
L = snman.lane_graph.create_lane_graph(G, lanes_attribute=KEY_LANES_DESCRIPTION_AFTER)
snman.io.export_lane_geometries(
    L,
    os.path.join(export_path, 'L_rebuilt_edges.gpkg'),
    os.path.join(export_path, 'L_rebuilt_nodes.gpkg')
)

In [ ]:
L = snman.io.load_lane_graph(
    os.path.join(export_path, 'steps', '6_L_edges.gpkg'),
    os.path.join(export_path, 'steps', '6_L_nodes.gpkg')
)

In [ ]:
H = copy.deepcopy(G)

street_graph.lane_graph_to_street_graph(
    H, L,
    KEY_LANES_DESCRIPTION_AFTER
)

snman.io.export_HLA(
    os.path.join(export_path, 'steps'),
    '7',
    H=H
)

In [ ]:
l = snman.space_allocation.Lane(LANETYPE_MOTORIZED, DIRECTION_FORWARD)
str(l)

In [ ]:
l.direction = DIRECTION_BOTH
str(l)

In [ ]:
snman.lane_graph.calculate_street_width(L, 4049, 7003, 0)